# w205 Project 2: Joshua Archer
## Contents

(1) Publish and consume messages with kafka

(2) Use Spark to transform messages

(3) Check out the json data

(4) Answer business questions

(5) Write to HDFS in parquet format for all created dataframes

(6) Relevant history from the console


In [1]:
import json
import pprint
from pyspark.sql import Row

## (1) Publish and consume messages with kafka

#### Copy in old yaml file

```cp ~/w205/course-content/06-Transforming-Data/docker-compose.yml ~/w205/project-2-jearcher/```

#### Copy in new YAML file (week 07)

```cp ~/w205/course-content/07-Sourcing-Data/docker-compose.yml ~/w205/project-2-jearcher/```

#### Make sure in right directory 

```cd ~/w205/project-2-jearcher/```

#### Spin up cluster

```docker-compose up -d```

#### Start the log file in 2nd command line (kafka mirror)

```docker-compose logs -f kafka```

#### Create a kafka topic called 'assessments'

```docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181```

#### Check/describe the topic

```docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181```

#### Publish messages to topic with kafka console producer (Change filename)

```docker-compose exec mids bash -c "cat /w205/project-2-jearcher/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments "```

#### Consume messages:

```docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"```


Alternate way to consume messages:

```docker-compose exec kafka kafka-console-consumer --bootstrap-server localhost:29092 --topic assessments --from-beginning --max-messages 3280```

#### Shut down cluster

```docker-compose down```

#### Check for any strays

```docker ps -a```

# (2) Use PySpark to transform messages

In [2]:
# Read from kafka using pyspark, subscribe to kafka topic and create a dataframe
assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

# Check out the schema
assessments.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [3]:
# Cast assessments as strings, store json converted to string in a new dataframe
assessments_as_strings=assessments.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Check it out
assessments_as_strings.show()
assessments_as_strings.printSchema()
assessments_as_strings.count()

+----+--------------------+
| key|               value|
+----+--------------------+
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
+----+--------------------+
only showing top 20 rows

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



3280

## (3) Check out / Unroll the json data


#### Start by checking out the json data

In [4]:
p = pprint.PrettyPrinter(indent=1)

In [5]:
f = open("assessment-attempts-20180128-121051-nested.json","r")

In [6]:
s = f.read()

In [7]:
json_data = json.loads(s)

In [8]:
f.close()

In [9]:
# Check out how many assessments there are
len(json_data) 

3280

In [10]:
# Pretty print json to look at it with human eyes and brain
# Helps to see all the different data stored here
p.pprint(json_data[0])

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
               'counts': {'all_correct': False,
                          'correct': 2,
                          'incomplete': 1,
                          'incorrect': 1,
                          'submitted': 4,
                          'total': 4,
                          'unanswered': 0},
               'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
               'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
                              'options': [{'at': '2018-01-23T14:23:24.670Z',
                                           'checked': True,
                                           'correct': True,
                                           'id': '

In [11]:
# Unroll json !! If Error: RDD EMPTY, make sure you published on kafka (part 1 of this markdown)
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

raw_assessments.cache()

assessments = raw_assessments.select(raw_assessments.value.cast('string'))

extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

extracted_assessments.registerTempTable('assessments')

spark.sql("select keen_id from assessments limit 10").show()

spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()

+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+

+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                         

In [12]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))

#### Simple query to check if it is working

In [13]:
spark.sql("select sequences.questions from assessments limit 10").show()

+--------------------+
|           questions|
+--------------------+
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
|[Map(user_incompl...|
+--------------------+



In [14]:
assessments_as_strings.select('value').take(1)

assessments_as_strings.select('value').take(1)[0].value



first_assessment=json.loads(assessments_as_strings.select('value').take(1)[0].value)

first_assessment

print(first_assessment['sequences'])

{'questions': [{'user_incomplete': True, 'user_correct': False, 'options': [{'checked': True, 'at': '2018-01-23T14:23:24.670Z', 'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d', 'submitted': 1, 'correct': True}, {'checked': True, 'at': '2018-01-23T14:23:25.914Z', 'id': 'f2528210-35c3-4320-acf3-9056567ea19f', 'submitted': 1, 'correct': True}, {'checked': False, 'correct': True, 'id': 'd1bf026f-554f-4543-bdd2-54dcf105b826'}], 'user_submitted': True, 'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986', 'user_result': 'missed_some'}, {'user_incomplete': False, 'user_correct': False, 'options': [{'checked': True, 'at': '2018-01-23T14:23:30.116Z', 'id': 'a35d0e80-8c49-415d-b8cb-c21a02627e2b', 'submitted': 1}, {'checked': False, 'correct': True, 'id': 'bccd6e2e-2cef-4c72-8bfa-317db0ac48bb'}, {'checked': True, 'at': '2018-01-23T14:23:41.791Z', 'id': '7e0b639a-2ef8-4604-b7eb-5018bd81a91b', 'submitted': 1, 'correct': True}], 'user_submitted': True, 'id': 'bbed4358-999d-4462-9596-bad5173a6ecb', 'user_result

#### Create lambda transform to start creating dataframes, temptables and queries

In [15]:
# extract sequence.id with lambda transform
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)

# create separate dataframe
my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()

# Register as a temp table for SQL
my_sequences.registerTempTable('sequences')

# Do some simple queries to check the data
spark.sql("select sequences_id from sequences limit 10").show()

spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show()

+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|        sequences_id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a26ee9cbf5ce1000...|1512500892.4166169|9bd87823-4508-4e0...|
|5a29dcac74b662000...|1512692908.8423469|e7110aed-0d08-4cb...|
|5a2fdab0eabeda000...|1513085616.2275269|cd800e92-afc3-447...|
|5a30105020e9d4000...|1513099344.8624721|8ac691f8-8c1a-403...|
|5a3a6fc3f0a100000...| 1513779139.354213|e7110aed-0d08-4cb...|
|5a4e17fe08a892000...|1515067390.1336551|9abd5b51-6bd8-11e...|
|5a4f3c69cc6444000...| 1515142249.858722|083844

In [16]:
# Create another dataframe questions
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list

my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()

my_questions.registerTempTable('questions')

spark.sql("select id, my_count from questions limit 10").show()

spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a26ee9cbf5ce1000...|1512500892.4166169|0603e6f4-c3f9-4c2...|
|5a26ee9cbf5ce1000...|1512500892.4166169|26a06b

In [17]:
# create another data frame correct total
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()

my_correct_total.registerTempTable('ct')

# Some queries
spark.sql("select * from ct limit 10").show()

spark.sql("select correct / total as score from ct limit 10").show()

spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()

spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()


+-------+-----+
|correct|total|
+-------+-----+
|      2|    4|
|      1|    4|
|      3|    4|
|      2|    4|
|      3|    4|
|      5|    5|
|      1|    1|
|      5|    5|
|      4|    4|
|      0|    5|
+-------+-----+

+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+

+-----------------+
|        avg_score|
+-----------------+
|62.65699745547047|
+-----------------+

+-------------------+
| standard_deviation|
+-------------------+
|0.31086692286170553|
+-------------------+



# (4) Answer Business questions
The following questions can be answered with the queries offered after each question:

1) What are the top 10 most taken classes?

In [18]:
# Explore exam names
# spark.sql('select exam_name from assessments').show

# Query for top 10 classes
spark.sql('select exam_name, count(*) as number_of_students from assessments group by exam_name order by number_of_students desc').show(10)

+--------------------+------------------+
|           exam_name|number_of_students|
+--------------------+------------------+
|        Learning Git|               394|
|Introduction to P...|               162|
|Introduction to J...|               158|
|Intermediate Pyth...|               158|
|Learning to Progr...|               128|
|Introduction to M...|               119|
|Software Architec...|               109|
|Beginning C# Prog...|                95|
|    Learning Eclipse|                85|
|Learning Apache M...|                80|
+--------------------+------------------+
only showing top 10 rows



2) What are the bottom 10, least taken classes?

In [19]:
# Bottom ten classes
spark.sql('select exam_name, count(*) as number_of_students from assessments group by exam_name order by number_of_students asc').show(10)


+--------------------+------------------+
|           exam_name|number_of_students|
+--------------------+------------------+
|Operating Red Hat...|                 1|
|Native Web Apps f...|                 1|
|Learning to Visua...|                 1|
|Nulls, Three-valu...|                 1|
|The Closed World ...|                 2|
|Arduino Prototypi...|                 2|
|What's New in Jav...|                 2|
|Hibernate and JPA...|                 2|
|Learning Spring P...|                 2|
|Understanding the...|                 2|
+--------------------+------------------+
only showing top 10 rows



# (5) Write to HDFS in parquet format for all created dataframes

In [20]:
raw_assessments.write.mode('overwrite').parquet("/tmp/raw_assessments")
assessments.write.mode('overwrite').parquet("/tmp/assessments")
my_questions.write.mode('overwrite').parquet("/tmp/questions")
my_sequences.write.mode('overwrite').parquet("/tmp/sequences")
my_correct_total.write.mode('overwrite').parquet("/tmp/sequences")

# (6) Relevant history from the console

```
  952  cd ~/w205/project-2-jearcher/
  953  docker-compose up -d
  954  docker-compose logs -f kafka
  955  docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
  956  docker-compose exec mids bash -c "cat /w205/project-2-jearcher/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments "
  957  docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"
  958  history > josh-archer-history.txt
  959  git status
  960  git add .
  961  git commit -m "forgot to commit for a while, mostly finished with proj2"
  962  git push origin assignment
  963  ls -lah
  964  vi .gitignore
  965  ls -lah
  966  git add .
  967  git commit -m "adding to gitignore to only leave behind files for submission"
  968  git push origin assignment
  969  git pull -all
  970  git pull
  971  git log
  972  git status
  973  ls -lah
  974  rm .~derby.log
  975  git status
  976  git add .
  977  git commit -m signing off to teach, nothing much added"
  978  git commit -m "signing off to teach, nothing much added"
  979  git push origin assignment
  980  history > josh-archer-history.txt
  ```